# BUILD DATASET
In this script we analyze the whole dataset

In [31]:
import csv
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import pickle
import calendar
import os
from datetime import datetime, timedelta


base_dir = '../ResOpsUS/'
logs_dir = '../logs/ResOpsUS/'

### Number of dams

In [2]:
data = pd.read_csv('../ResOpsUS/' + "time_series_inventory.csv", delimiter=',')


In [3]:
len(data)

678

### Number of dams where both STORAGE and OUTFLOW is recorded

In [4]:
data = data[(data['STORAGE'] == 1) & (data['OUTFLOW'] == 1)]

In [5]:
len(data)

506

### We switch to the water year and we save information for day-month-year of each observation

### We handle years with the 29th of February by merging it to the 1st of March
### We keep only years with more than 330 POSITIVE observation of both STORAGE and OUTFLOW
### We perform outlier detection on the remaining data and then we impute all the missing values through a simple spline interpolation

In [38]:
idxs = data["DAM_ID"]

In [39]:
actual_length = []
orig_dir = '../ResOpsUS/time_series_all/'
save_dir = '../ResOpsUS/time_series/'


for idx in idxs:

    df = pd.read_csv(orig_dir + "ResOpsUS_" + str(idx) + ".csv", delimiter=',')
    df = df[["date","outflow","storage"]]
    
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['water_year'] = df['year']
    df.loc[(df['month'] == 10) | (df['month'] == 11) | (df['month'] == 12),"water_year"] += 1

    start_year = df['water_year'].iloc[0]
    end_year = df['water_year'].iloc[-1]
    
    n_days = df.groupby('water_year').count()['day'].values
    n_years = end_year + 1 - start_year
    
    # Remove negative values of outflow

    df.loc[df["outflow"] < 0, 'outflow'] = np.nan 
    
    #### 29 FEB HANDLING
    
    for i, y in enumerate(range(start_year, end_year)):
        _29_feb = df[(df['year'] == y) & (df['month'] == 2) & (df['day'] == 29)]
        _1_mar = df[(df['year'] == y) & (df['month'] == 3) & (df['day'] == 1)]

        if _29_feb['outflow'].values.size == 0 or _1_mar['outflow'].values.size == 0:
            continue

        new_outflow = (_29_feb['outflow'].values + _1_mar['outflow'].values) / 2

        df.at[(df['year'] == y) & (df['month'] == 3) & (df['day'] == 1), 'outflow'] = new_outflow

    df = df[~((df['month'] == 2) & (df['day'] == 29))]
    
    #### MORE THAN 330 VALUES
    
    period = df.groupby('water_year').count()[['storage','outflow']].values
    period_mask = np.zeros(len(period), dtype=bool)
    period_mask[(period[:,0] > 330) & (period[:,1] > 330)] = 1

    valid_years = np.array(range(start_year, end_year + 1))[period_mask]

    df = df.loc[df['water_year'].isin(valid_years)]
    
    if len(df) == 0:
        actual_length.append(0)
        continue
    
    #### n_days
    
    df["n_day"] = np.zeros(len(df["date"]))

    for i in range(len(df["date"])):
        if calendar.isleap(df["date"].iloc[i].year) and df["date"].iloc[i].timetuple().tm_yday > 59: 
            df["n_day"].iloc[i] = df["date"].iloc[i].timetuple().tm_yday - 1
        else:
            df["n_day"].iloc[i] = df["date"].iloc[i].timetuple().tm_yday
            
    #### OUTLIER DETECTION
    
    outlier_df = df["storage"].values
    window_size = 5

    window_mean = np.zeros_like(outlier_df)

    left_filler = [outlier_df[0]] * int(((window_size - 1) / 2))
    right_filler = [outlier_df[-1]] * int(((window_size - 1) / 2))

    outlier_df = np.concatenate((left_filler, outlier_df, right_filler))

    for i in range(len(window_mean)):
        window_mean[i] = np.nanmedian(outlier_df[i:i + window_size])
        
    res = window_mean - outlier_df[2:-2]
    
    res_nan = res[~np.isnan(res)]
    res_nan_not0 = res_nan[res_nan != 0]

    q_10 = np.quantile(res_nan_not0, 0.1)
    q_90 = np.quantile(res_nan_not0, 0.9)

    IQR = q_90 - q_10

    outlier_mask = np.zeros(len(res), dtype = bool)
    outlier_mask[abs(res) > 3 * IQR] = 1

    for i in range(len(outlier_mask)):
        if outlier_mask[i]:
            total_var = abs(df["storage"].iloc[i-1] - df["storage"].iloc[i]) + abs(df["storage"].iloc[i+1] - df["storage"].iloc[i]) 
            delta = abs(df["storage"].iloc[i-1] - df["storage"].iloc[i+1])

            if total_var > 3 * delta:
                df["storage"].iloc[i] = np.NaN
    
    #### MISSING VALUES IMPUTATION
    
    df["storage"] = df["storage"].interpolate(method='spline', order = 3)
    df["outflow"] = df["outflow"].interpolate(method='spline', order = 3)
    
    df = df[df["storage"].notnull() & df["outflow"].notnull()]
    
    if len(df[df["water_year"] == df['water_year'].iloc[0]]) != 365:
        df = df[df["water_year"] > df['water_year'].iloc[0]]
        
    if len(df[df["water_year"] == df['water_year'].iloc[-1]]) != 365:
        df = df[df["water_year"] < df['water_year'].iloc[-1]]
    
    actual_length.append(int(len(df)/365))
    
    df.to_csv(save_dir + "ResOpsUS_" + str(idx) + ".csv")
    
   

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid val

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWa

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

...

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set o

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipyke

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: Settin

/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/Manuel/.virtualenvs/irl_real_life/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: Run